In [27]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi

import sys
sys.path.insert(0, '../')



In [28]:
from config_paper import key_id, secret_key
from cross_signal import *

In [29]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(key_id, secret_key,base_url )

In [31]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-04-26'

In [46]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['ema_high'] = tab.EMA(dataframe, timeperiod=10, price='high')
        dataframe['ema_close'] = tab.EMA(dataframe, timeperiod=10, price='close')
        dataframe['ema_low'] = tab.EMA(dataframe, timeperiod=10, price='low')
        
        
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        stoch_fast = tab.STOCHF(dataframe, 10.0, 3.0, 0.0, 3.0, 0.0)
        dataframe['fastd'] = stoch_fast['fastd']
        dataframe['fastk'] = stoch_fast['fastk']
        dataframe['adx'] = tab.ADX(dataframe)
        dataframe['cci'] = tab.CCI(dataframe)
        # required for graphing

        return dataframe

In [32]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'400000'

In [59]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    x = f.read().split('\n')

In [60]:
x

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AAP',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'AGN',
 'ADS',
 'LNT',
 'ALL',
 'GOOGL',
 'MO',
 'AMZN',
 'AMC',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ARNC',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BDX',
 'BRKb',
 'BBY',
 'BIIB',
 'BLK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BFb',
 'CHRW',
 'COG',
 'CDNS',
 'CFB',
 'COF',
 'CPRI',
 'CAH',
 'KMX',
 'CCL',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSa',
 'CMA',
 'CAG',
 

In [61]:
api = tradeapi.REST(key_id ,secret_key )

barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = x
positionSizing = 0.25

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)


In [83]:
returned_data = api.polygon.historic_agg_v2('FLS', 1, 'minute',_from='2020-04-23', to=startdate,).df
returned_data

,open,high,low,close,volume
timestamp,,,,,
2020-04-23 08:43:00-04:00,25.3299,25.330,25.3299,25.330,200
2020-04-23 08:49:00-04:00,24.6600,24.660,24.6600,24.660,220
2020-04-23 09:30:00-04:00,24.9700,25.290,24.8700,25.290,13814
2020-04-23 09:32:00-04:00,25.0750,25.075,25.0750,25.075,125
2020-04-23 09:33:00-04:00,24.8900,25.280,24.8900,24.910,410
...,...,...,...,...,...
2020-04-24 15:58:00-04:00,25.5900,25.600,25.5500,25.575,22194
2020-04-24 15:59:00-04:00,25.5700,25.570,25.5000,25.520,49956
2020-04-24 16:02:00-04:00,25.5300,25.530,25.5300,25.530,277378


In [82]:
returneddata = api.get_barset('AMP',barTimeframe,limit=150).df
data_indic=populateindicators(returneddata['AMP'])


C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [86]:
datalist=[]
e=0
for i in range(assetListLen) :
    try:
        symbol = assetsToTrade[i]

        returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from=startdate, to=startdate,).df
        #api.get_barset(symbol,barTimeframe,limit=150).df


        # Calculated trading indicators
        #populate exponential moving averages
        data=populateindicators(returned_data) 

        datalist.append(data)
         # Create the buy signal here
        if data['adx'][-1] > 22 and \
        data['mfi'][-1] < 25 and \
        data['ultosc'][-1] < 25 and \
        data['rsi'][-1] < 33 and \
        data['minusdi'][-1] > 25 and \
        data['minusdi'][-1] > data['plusdi'][-1]:


            openPosition = api.get_position(symbol)

            # Opens new position if one does not exist
            if openPosition == 0:
                cashBalance = api.get_account().cash

                targetPositionSize = cashBalance / (price / positionSizing) # Calculates required position size

                returned = api.submit_order(symbol,targetPositionSize,"buy","market","gtc") # Market order to open position
                print(returned)


        elif data['adx'][-1] > 22 and \
        data['mfi'][-1] > 63 and \
        data['rsi'][-1] > 75 and \
        data['plusdi'][-1] > 25 and \
        data['minusdi'][-1] < data['plusdi'][-1]:
            # Closes position  SMA50
            openPosition = api.get_position(symbol)

            returned = api.submit_order(symbol,openPosition,"sell","market","gtc") # Market order to fully close position
            print(returned)

    except:
            e=e+1

In [87]:
e

501

In [84]:
datalist[10]

,open,high,low,close,volume,ema_high,ema_close,ema_low,mfi,ultosc,minusdi,plusdi,rsi,fastd,fastk,adx,cci
timestamp,,,,,,,,,,,,,,,,,
2020-04-23 08:49:00-04:00,12.680,12.680,12.680,12.680,1078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-23 09:30:00-04:00,12.820,12.900,12.760,12.860,80654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-23 09:31:00-04:00,12.840,12.840,12.740,12.740,1733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-23 09:32:00-04:00,12.730,12.820,12.720,12.810,7378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-23 09:33:00-04:00,12.820,12.860,12.810,12.810,3167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-24 15:56:00-04:00,12.830,12.865,12.830,12.840,54590,12.892332,12.871258,12.865616,19.401283,20.487746,31.119855,9.407323,30.404588,4.421949,9.099181,39.010341,-87.866822
2020-04-24 15:57:00-04:00,12.840,12.845,12.815,12.845,72789,12.883726,12.866483,12.856413,17.182920,31.785184,32.758046,8.591069,31.969414,11.039465,24.019215,40.398615,-91.923649
2020-04-24 15:58:00-04:00,12.840,12.850,12.820,12.830,40154,12.877594,12.859850,12.849793,16.634869,33.887295,29.958639,9.281186,29.849947,15.387292,13.043478,41.276933,-86.815560
